# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

In [6]:
establishments.find_one()

{'_id': ObjectId('64f0ddef04e1b1fe7b6b9897'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': 4,
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCo

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1, '_id': 0}

results = list(establishments.find(query, fields))

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(results[0])

41
{'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [8]:
# Convert the result to a Pandas DataFrame
results_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
pprint(results_df.shape[0])
# Display the first 10 rows of the DataFrame
# pprint(results_df.head(10))
results_df.head(10)

41


,BusinessName,scores.Hygiene
0,The Chase Rest Home,20
1,Brenalwood,20
2,Melrose Hotel,20
3,Seaford Pizza,20
4,Golden Palace,20
5,Ashby's Butchers,20
6,South Sea Express Cuisine,20
7,Golden Palace,20
8,The Tulip Tree,20
9,F & S,20


In [9]:
#ADDED CELL
#1. Which establishments have a hygiene score equal to 20?
# Answer: see the full list below
results_df

,BusinessName,scores.Hygiene
0,The Chase Rest Home,20
1,Brenalwood,20
2,Melrose Hotel,20
3,Seaford Pizza,20
4,Golden Palace,20
5,Ashby's Butchers,20
6,South Sea Express Cuisine,20
7,Golden Palace,20
8,The Tulip Tree,20
9,F & S,20


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte':4}}
results2 = list(establishments.find(query, fields))
fields = {'LocalAuthorityName':1,
         'RatingValue':1, '_id': 0}
# Use count_documents to display the number of documents in the result
pprint(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results2[0])

33
{'BusinessName': "Charlie's", 'scores': {'Hygiene': 5}}


In [11]:
# Convert the result to a Pandas DataFrame
results2_df = pd.json_normalize(results2)
# Display the number of rows in the DataFrame
pprint(results2_df.shape[0])
# Display the first 10 rows of the DataFrame
results2_df.head(10)

33


,BusinessName,scores.Hygiene
0,Charlie's,5
1,Mv City Cruises Erasmus,0
2,Benfleet Motor Yacht Club,0
3,Coombs Catering t/a The Lock and Key,0
4,Tilbury Seafarers Centre,0
5,Mv Valulla,0
6,Tereza Joanne,0
7,Brick Lane Brews,10
8,The Nuance Group (UK) Limited,0
9,City Bar & Grill,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Get Penang Flavours document
query_penang_flavours = {'BusinessName': "Penang Flavours"}
penang_result = establishments.find_one(query_penang_flavours)

degree_search = 0.01
latitude = penang_result['geocode']['latitude']
longitude = penang_result['geocode']['longitude']

query = {'RatingValue': 5,
            'geocode.latitude': {
                '$gte': latitude - degree_search, 
                '$lte': latitude + degree_search
                },
            'geocode.longitude': {
                '$gte': longitude - degree_search, 
                '$lte': longitude + degree_search
             }
        }
fields = {'RatingValue': 1, 
          'geocode.latitude': 1, 
          'geocode.longitude': 1, 
          'scores.Hygiene': 1,
          'BusinessName': 1,
          '_id':0}

sort =  [('scores.Hygiene', 1)]

results = list(establishments.find(query, fields).sort(sort).limit(5))
# Print the results
pprint(results)

[{'BusinessName': 'Volunteer',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4873437, 'longitude': 0.09208},
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Plumstead Manor Nursery',
  'RatingValue': 5,
  'geocode': {'latitude': 51.481517791748, 'longitude': 0.0859939977526665},
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Atlantic Fish Bar',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4867296, 'longitude': 0.0912164},
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Iceland',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4871482849121, 'longitude': 0.0924199968576431},
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
  'scores': {'Hygiene': 0}}]


In [13]:
# Convert result to Pandas DataFrame
top5_london_df = pd.json_normalize(results)
pprint(top5_london_df.shape[0])
top5_london_df.head()

5


,BusinessName,RatingValue,scores.Hygiene,geocode.longitude,geocode.latitude
0,Volunteer,5,0,0.092080,51.487344
1,Plumstead Manor Nursery,5,0,0.085994,51.481518
2,Atlantic Fish Bar,5,0,0.091216,51.486730
3,Iceland,5,0,0.092420,51.487148
4,Howe and Co Fish and Chips - Van 17,5,0,0.092537,51.487534


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {
                    'scores.Hygiene': 0}
              }

# Print the number of documents in the result
group_query = {'$group': {
    '_id':{
        'LocalAuthorityName': '$LocalAuthorityName'
    },
    'count':{'$sum': 1}
  }
}
sort_values = {'$sort': {'count': -1}}
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results[:10])

[{'_id': {'LocalAuthorityName': 'Thanet'}, 'count': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'count': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'count': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'count': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'count': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'count': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'count': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'count': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'count': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'count': 542}]


In [15]:
# Convert the result to a Pandas DataFrame
results_df = pd.json_normalize(results)
results_df = results_df.rename(columns={'_id.LocalAuthorityName':'_id'})
# Display the number of rows in the DataFrame
pprint(results_df.shape[0])
# Display the first 10 rows of the DataFrame
results_df.head(10)

55


,count,_id
0,1130,Thanet
1,882,Greenwich
2,713,Maidstone
3,711,Newham
4,686,Swale
5,680,Chelmsford
6,672,Medway
7,607,Bexley
8,586,Southend-On-Sea
9,542,Tendring
